In [24]:
import pandas as pd
from pandas_datareader import data as pdr

import numpy as np

import matplotlib.pyplot as plt
import datetime as dt

import scipy as sci
from scipy import stats
import scipy.optimize as sco

import yfinance as yf

from pypfopt import(
    EfficientFrontier,
    risk_models,
    expected_returns
)
from pypfopt.discrete_allocation import(
    DiscreteAllocation,
    get_latest_prices
)

import plotly.graph_objects as go

In [25]:
# specifiy our date range for data
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days = 1825)
start, end = start_date, end_date
start, end                                       # verify our date range

(datetime.datetime(2017, 10, 14, 14, 51, 39, 398141),
 datetime.datetime(2022, 10, 13, 14, 51, 39, 398141))

In [26]:
# Basket
asset_list = ['AAPL', 'IBM', 'TSLA', 'GOOGL']
stocks = ['^GSPC'] + asset_list                              # adding S&P 500 index (^GSPC)for use in beta weighting
stocks

['^GSPC', 'AAPL', 'IBM', 'TSLA', 'GOOGL']

In [27]:
# get the data
def get_the_data(stocks, start, end):
    stock_data = pdr.get_data_yahoo(stocks, start, end)
    stock_data = stock_data['Close']
    returns = stock_data.pct_change()
    mean_returns = returns.mean()
    log_returns = np.log(stock_data / stock_data.shift(1)).dropna()
    stock_covariance = returns.cov()
    return stock_data, mean_returns, log_returns, stock_covariance

stock_data, mean_returns, log_returns, stock_covariance = get_the_data(stocks, start, end)

In [28]:
# Read in price data
# df = pd.read_csv("tests/resources/stock_prices.csv", parse_dates=True, index_col="date")

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(stock_data)
S = risk_models.sample_cov(stock_data)

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file('weights.csv')  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('^GSPC', 0.0), ('AAPL', 0.64523), ('IBM', 0.0), ('TSLA', 0.35477), ('GOOGL', 0.0)])
Expected annual return: 39.0%
Annual volatility: 37.5%
Sharpe Ratio: 0.99


(0.39030140937381186, 0.3749573956034909, 0.9875826259615835)

In [29]:
latest_prices = get_latest_prices(stock_data)

da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'AAPL': 45, 'TSLA': 16}
Funds remaining: $8.54


In [30]:
# performance
def portfolio_performance(weight, mean_returns, covariance, hurdle_rate = 0.0275):
    returns = np.sum(mean_returns * weight)*252   # (mean_returns * weight).sum()*252
    std_dev = np.sqrt(np.dot(weight.T, np.dot(covariance, weight))) * np.sqrt(252)    #  portfolio variance
    return returns, std_dev


In [31]:
def negative_sharpe_rat(weight, mean_returns, covariance, hurdle_rate = 0.0275):
    p_returns, p_standard_dev = portfolio_performance(weight, mean_returns, covariance)
    return -1*(p_returns - hurdle_rate) / p_standard_dev                

In [32]:
def optimal_sharpe_rat(mean_returns, covariance, hurdle_rate=0.03, boundaries_set = (0,1)):
    "maximize sharpe rat or minimize negative sharpe rat via portfolio weighting"
    number_securities = len(mean_returns)
    args = (mean_returns, covariance, hurdle_rate)
    boundaries = ({'type': 'eq', 'fun': lambda x: np.sum(x) -1})     # see np scipy 'SLSQP', this is ~lambda weight
    reaches = boundaries_set
    barriers = tuple(reaches for security in range(number_securities))
    result = sco.minimize(negative_sharpe_rat(weight, number_securities* [1.0/number_securities], args = args, method = 'SLSQP', barriers = barriers, boundaries = boundaries))   
    return result


In [33]:
def portfolio_variance(weight, mean_returns, covariance):
    return portfolio_performance(weight, mean_returns, covariance)[1]

In [34]:
def minimize_variance(mean_returns, covariance, boundaries_set = (0,1)):
    "maximize volatility through adjusting security weighting in portfolio"
    number_securities = len(mean_returns)
    args = (mean_returns, covariance)
    boundaries = ({'type': 'eq', 'fun': lambda x: np.sum(x) -1})
    reaches = boundaries_set
    barriers = tuple(reaches for security in range(number_securities))
    result = sco.minimize(portfolio_variance, number_securities* [1.0/number_securities], args = args,
                                   method = 'SLSQP', barriers = barriers, boundaries = boundaries)   
    return result

In [35]:
def computed_results(mean_returns, covariance, hurdle_rate=0.03, boundaries_set = (0,1)):
                          
    # Optimal Sharpe Ratio Portfolio
    portfolio_optimal_sharpe_rat = optimal_sharpe_rat(mean_returns, covariance)
    returns_optimal_sharpe_rat, standard_dev_optimal_sharpe_rat = portfolio_performance(portfolio_optimal_sharpe_rat['x'], mean_returns, covariance) 
    allocation_optimal_sharpe_rat = pd.DataFrame(portfolio_optimal_sharpe_rat['x'], index=mean_returns.index, columns=['allocation'])
    allocation_optimal_sharpe_rat.allocation = [round(i*100, 0) for i in allocation_optimal_sharpe_rat.allocation]                      
                          
    # Least Risk Portfolio                      
    volume_limited_portfolio = minimize_variance(mean_returns, covariance) 
    returns_volume_limited, standard_dev_volume_limited = portfolio_performance(volume_limited['x'], mean_returns, covariance)   
    allocation_volume_limited = pd.DataFrame(volume_limited_portfolio['x'], index=mean_returns.index, columns=['allocation']) 
    allocation_volume_limited.allocation = [round(i*100, 0) for i in allocation_optimal_sharpe_rat.allocation]                      
    
    # Efficient Frontier
    efficient_composition = []
    target_returns = np.linspace(returns_volume_limited, returns_optimal_sharpe_rat, 20)
    for target in target_returns:
        efficient_composition.append(efficient_opt(mean_returns, covariance, target,['fun']))   
    
    returns_optimal_sharpe_rat, standard_dev_optimal_sharpe_rat = round(returns_optimal_sharpe_rat*100,2), round(standard_dev_optimal_sharpe_rat*100,2)                      
    returns_volume_limited, standard_dev_volume_limited = round(returns_volume_limited*100, 2), round(standard_dev_volume_limited*100, 2)                      
                          
    return returns_optimal_sharpe_rat, 
    standard_dev_optimal_sharpe_rat, 
    allocation_optimal_sharpe_rat, 
    returns_volume_limited, 
    standard_dev_volume_limited, 
    allocation_volume_limited, 
    efficient_composition, target_returns              

In [36]:
def ef_plot(mean_returns, covariance, hurdle_rate=0.0275, boundaries_set = (0,1)):
    returns_optimal_sharpe_rat, standard_dev_optimal_sharpe_rat, allocation_optimal_sharpe_rat, returns_volume_limited, standard_dev_volume_limited, allocation_volume_limited, efficient_composition, target_returns = computed_results(mean_returns, covariance, hurdle_rate, boundaries_set)                   
                          
    # Optimal Sharpe Ratio
    optimal_sharpe_rat = go.scatter(
         name = 'Optimal Sharpe Ratio',
         mode = 'markers',
         x = [standard_dev_optimal_sharpe_rat],
         y = [returns_optimal_sharpe_rat],
         marker = dict(color='magenta', size=15, line=dict(width=4, color='blue'))
    )
                          
    # Least Risk Volume
    limited_volume = go.scatter(
         name = 'Minimum Risk',
         mode = 'markers',
         x = [standard_dev_volume_limited],
         y = [returns_volume_limited],
         marker = dict(color='red', size=15, line=dict(width=4, color='blue'))
    )                          
                          

    # Efficient Frontier
    graph_ef = go.scatter(
         name = 'Efficient Frontier',
         mode = 'liness',
         x = [round(ef_std * 100, 2) for ef_std in efficient_composition],
         y = [round(target * 100, 2) for target in target_returns],
         line = dict(color='black', width=3, dash = 'dotdash' color='blue')
    )                          
                                   
                          
    traces = ['optimal_sharpe_rat', 'limited_volume', 'graph_ef']                     
    configuration = go.layout(
         title = 'Modern Portfolio Thoery - Optimization with Efficient Frontier',
         yaxis = dict(title = 'Percent Annual Return'),
         xaxis = dict(title ='Percent Annual Volatility'),
         showlegend = True,
         legend = dict(
             x = 0.75, y = 0, traceorder = 'normal',
             bgcolor = '#E2E2E2',                           # gray background
             bordercolor = 'black',
             borderwidth = 2),
         width = 800,
         height = 600)

                          
                          
    fig = go.figure(data=traces, layout = configuration)                    
    return fig.show()                     
                          
                          
                          
                          
stocklist = ['GOOGL', 'AAPL', 'TSLA', 'IBM']
stocks = stocklist

end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=365)

weight = np.array([0.3, 0.3, 0.3, 0.1])

mean_returns, covariance = gets_the_data(stocks, start = start_date, end = end_date)
#mean_returns, covariance = gets_the_data(stocks, start = start_date, end = end_date)
returns, std_dev = portfolio_performance(weight, mean_returns, covariance)

                       
                          
# result = optimal_sharpe_rat(mean_returns, covariance, weight, hurdle_rate=0.03)  
result =  optimal_sharpe_rat(mean_returns, covariance)   
opt_sharpe, opt_weight = result['fun'], result['x']                           
print(opt_sharpe, opt_weight)  
                          
                          
                          
                          
 
minimal_var_result= minimize_variance(mean_returns, covariance)
min_var, min_var_weight = minimal_var_result['fun'], minimal_var_result['x']
          
print(returns, std_dev)
          

ef_plot(mean_returns, covariance)







SyntaxError: invalid syntax (704352117.py, line 29)